In [66]:
!pip install transformers torch
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [78]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [79]:
dir_rawData_id = 66
dir_processData_id = 67

In [80]:
datanode = DataNode()

Init DataNode sucessful!


In [81]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_rawData_id, name='Temp.json'))
meta

{'owner': 'nutcha',
 'human_size': '432.21 KB',
 'name': 'Temp.json',
 'description': 'file Temp.json',
 'directory': 66,
 'path': 'nutcha/DSI310-11/rawData/Temp.json'}

In [82]:
data = json.load(fp)
data

[{'_id': 1,
  'Year': 2019,
  'Month': 1,
  'ProvinceID': 10,
  'ProvinceName': 'กรุงเทพมหานคร',
  'Tmax': 37.3,
  'Tmin': 18.7},
 {'_id': 2,
  'Year': 2019,
  'Month': 1,
  'ProvinceID': 11,
  'ProvinceName': 'สมุทรปราการ',
  'Tmax': 32.2,
  'Tmin': 23.9},
 {'_id': 3,
  'Year': 2019,
  'Month': 1,
  'ProvinceID': 13,
  'ProvinceName': 'ปทุมธานี',
  'Tmax': 36.5,
  'Tmin': 20},
 {'_id': 4,
  'Year': 2019,
  'Month': 1,
  'ProvinceID': 14,
  'ProvinceName': 'พระนครศรีอยุธยา',
  'Tmax': 36.5,
  'Tmin': 18.5},
 {'_id': 5,
  'Year': 2019,
  'Month': 1,
  'ProvinceID': 16,
  'ProvinceName': 'ลพบุรี',
  'Tmax': 35.5,
  'Tmin': 16.2},
 {'_id': 6,
  'Year': 2019,
  'Month': 1,
  'ProvinceID': 18,
  'ProvinceName': 'ชัยนาท',
  'Tmax': 34.7,
  'Tmin': 18},
 {'_id': 7,
  'Year': 2019,
  'Month': 1,
  'ProvinceID': 20,
  'ProvinceName': 'ชลบุรี',
  'Tmax': 36.8,
  'Tmin': 18},
 {'_id': 8,
  'Year': 2019,
  'Month': 1,
  'ProvinceID': 21,
  'ProvinceName': 'ระยอง',
  'Tmax': 34.1,
  'Tmin': 20.2},


In [83]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [84]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [85]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-PER', 'score': 0.9993357, 'index': 4, 'word': '▁Wolfgang', 'start': 10, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9990395, 'index': 9, 'word': '▁Berlin', 'start': 33, 'end': 40}]


In [86]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [87]:
df = pd.DataFrame(data)
df

,_id,Year,Month,ProvinceID,ProvinceName,Tmax,Tmin
0,1,2019,1,10,กรุงเทพมหานคร,37.3,18.7
1,2,2019,1,11,สมุทรปราการ,32.2,23.9
2,3,2019,1,13,ปทุมธานี,36.5,20.0
3,4,2019,1,14,พระนครศรีอยุธยา,36.5,18.5
4,5,2019,1,16,ลพบุรี,35.5,16.2
...,...,...,...,...,...,...,...
2855,2856,2022,8,92,ตรัง,35.3,23.0
2856,2857,2022,8,93,พัทลุง,35.5,23.7
2857,2858,2022,8,94,ปัตตานี,35.1,23.1
2858,2859,2022,8,95,ยะลา,35.5,22.4


In [88]:
datanode.write(df=df, directory=dir_processData_id, name="data", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_rawData_id, name='Temp.json')])

{'sucess': True,
 'file_id': 97,
 'path': 'nutcha/DSI310-11/processData/data.parquet',
 'lineage': True}